In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.interpolate import interp1d

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Fitting from a sum of X gaussians
# f(x) = a * exp( -\frac{(x - b)^2}{2c^2} )
def vargauss(x, *args):
    count = int(len(args)/3)
    # need center, width and scaling
    scale = args[0::3]
    center = args[1::3]
    width = args[2::3]
    y = np.zeros_like(x)
    for m in range(count):
        s = scale[m]
        c = center[m]
        w = width[m]
        y = y + s*np.exp(-(((x - c))/w)**2)
    return y

In [ ]:
# read the Gaussian patterns and fit parameters

# read in the torsion patterns
fit_file = "gauss-fit.txt"
new_file = "qtdg.txt"

with open(new_file, "w") as fits:
    fits.write("// qtdg patterns\n")

with open(fit_file) as patterns:
    pattern = 0
    for line in patterns:
        pattern += 1
        # skip the comment at the top
        if "//" in line:
            continue
            
        smarts = line.split()[0]
        args = np.array(line.split()[1:], dtype=np.float32)
        
        x = np.linspace(0.0, 360.0, 360)
        # build up the y potential in a loop
        degrees = np.zeros_like(x)
        total = 0.0
        for d in range(360):
            degree = d
            if d > 180:
                degree = 180 - (d - 180)
            degrees[d] = float(degree)
            
        values = vargauss(degrees, *args)
        totals = np.cumsum(values)
        
        if (np.sum(values) < 0.01):
            print(smarts, "weird sum")
            continue
        elif (not np.isfinite(np.sum(values))):
            print(smarts, "nan")
            continue
        
        totals = totals / np.sum(values) # normalize
            
        # okay, now we need to invert these
        totals = np.insert(totals, 0, 0.0)
        x = np.insert(x, 0, 0.0)
        
        inverted = interp1d(totals, x)
        steps = np.arange(0.0, 1.0, 0.05)
        val = inverted(steps)
        # print(steps, val)
        
        # append the smarts and fits
        with open(new_file, "a") as fits:
            array = np.array2string(val, formatter={'float_kind':lambda x: "%.1f" % x}, separator=' ', max_line_width=511)
            fits.write(smarts + ' ' + array[1:-1] + '\n')